In [4]:
# Pandas
import pandas as pd
import sqlite3
import os
import datetime
import csv

# SQL Alchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date

# Define path to local sqlite database
database_path = "./EmployeeSQL/PewlettHackard.sqlite"

In [ ]:
# Create Classes (schema) - not needed if table created from pandas DataFrame!
# class Employees(Base):
#     __tablename__ = 'employees'

#     emp_no = Column(Integer, primary_key = True)
#     emp_title_id = Column(String(10))
#     birth_date = Column(Date)
#     first_name = Column(String(255))
#     last_name = Column(String(255))
#     sex = Column(String(1))
#     hire_date = Column(Date)

In [5]:
# Create an engine that can talk to the database, plus a database connection
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [6]:
# Create a "Metadata" layer to access the database
# ----------------------------------
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [7]:
# Create a Session Object to connect to the database to allow SQL queries
# ----------------------------------
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [24]:
# Load employees csv file into a pandas DataFrame converting dates to SQL-friendly format
custom_date_parser = lambda x: pd.to_datetime(x, format='%m/%d/%Y').date()
employees = pd.read_csv('./EmployeeSQL/employees.csv', parse_dates=['birth_date', 'hire_date'], date_parser=custom_date_parser)

In [25]:
# Display the employees DataFrame
employees.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date
0,473302,s0001,1953-07-25,Hideyuki,Zallocco,M,1990-04-28
1,475053,e0002,1954-11-18,Byong,Delgrande,F,1991-09-07
2,57444,e0002,1958-01-30,Berry,Babb,F,1992-03-21
3,421786,s0001,1957-09-28,Xiong,Verhoeff,M,1987-11-26
4,282238,e0003,1952-10-28,Abdelkader,Baumann,F,1991-01-18


In [26]:
# Add employees records to the employees tables
employees.to_sql('employees', engine, if_exists='replace', index=False)
session.commit()

In [29]:
# List the first name, last name, and hire date for the employees who were hired in 1986.
employee_hire = pd.read_sql("SELECT first_name, last_name, \
                       strftime('%Y-%m-%d', hire_date) AS hire_date \
                       FROM employees WHERE hire_date BETWEEN '1986-01-01' AND '1986-12-31' \
                       ORDER BY hire_date", conn)
employee_hire.head()

,first_name,last_name,hire_date
0,Vidya,Dymetman,1986-01-01
1,Shawna,Reinhart,1986-01-01
2,Abdelkader,Marrakchi,1986-01-01
3,Sumant,Stemann,1986-01-01
4,Danae,Nyanchama,1986-01-01
